# TensorFlow script mode training and serving

Script mode is a training script format for TensorFlow that lets you execute any TensorFlow training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native TensorFlow support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). In this example, we will show how easily you can train a SageMaker using TensorFlow 1.x and TensorFlow 2.x scripts with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker TensorFlow Serving container](https://github.com/aws/sagemaker-tensorflow-serving-container). The TensorFlow Serving container is the default inference method for script mode. For full documentation on the TensorFlow Serving container, please visit [here](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst).


# Set up the environment

Let's start by setting up the environment:

## Install SageMaker Python SDK with TensorFlow 2.x Support (>1.49)
_Note:  Ignore Warnings and Errors Below_

In [23]:
!pip install sagemaker --upgrade --ignore-installed --no-cache

    100% |████████████████████████████████| 296kB 35.6MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 57.8MB/s ta 0:00:01
    100% |████████████████████████████████| 20.1MB 110.2MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 118.1MB/s ta 0:00:01
    100% |████████████████████████████████| 26.1MB 108.5MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 69.8MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 77.9MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 111.4MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 91.0MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 68.0MB/s ta 0:00:01
    100% |████████████████████████████████| 122kB 74.2MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 114.2MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 116.2MB/s ta 0:00:01
    100% |████████████████████████████████| 552kB 111.6MB/s ta 0:00:01
    100% |████

In [24]:
!pip list

Package                            Version   
---------------------------------- ----------
absl-py                            0.8.1     
alabaster                          0.7.10    
anaconda-client                    1.6.14    
anaconda-project                   0.8.2     
asn1crypto                         0.24.0    
astor                              0.8.0     
astroid                            1.6.3     
astropy                            3.0.2     
attrs                              18.1.0    
Automat                            0.3.0     
autovizwidget                      0.13.1    
awscli                             1.16.283  
Babel                              2.5.3     
backcall                           0.1.0     
backports.shutil-get-terminal-size 1.0.0     
bcrypt                             3.1.7     
beautifulsoup4                     4.6.0     
bitarray                           0.8.1     
bkcharts                           0.2       
blaze                             

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Create the SageMaker Session

In [30]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

## Setup the Service Execution Role and Region

In [31]:
role = get_execution_role()
print('RoleARN:  {}\n'.format(role))
region = sagemaker_session.boto_session.region_name
print('Region:  {}'.format(region))

RoleARN:  arn:aws:iam::835319576252:role/service-role/AmazonSageMaker-ExecutionRole-20191006T135881

Region:  us-east-1


## Training Data

The MNIST dataset has been loaded to the public S3 buckets ``sagemaker-sample-data-<REGION>`` under the prefix ``tensorflow/mnist``.

In [32]:
original_training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)
print(original_training_data_uri)

s3://sagemaker-sample-data-us-east-1/tensorflow/mnist


In [39]:
local_data_path = './tensorflow-mnist/data'

## Copy the Training Data to Your S3 Bucket

In [40]:
!aws --region {region} s3 cp --recursive {original_training_data_uri} {local_data_path}

download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/eval_labels.npy to tensorflow-mnist/data/eval_labels.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_labels.npy to tensorflow-mnist/data/train_labels.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/eval_data.npy to tensorflow-mnist/data/eval_data.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_data.npy to tensorflow-mnist/data/train_data.npy


There are four ``.npy`` file under this prefix:
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [42]:
!ls {local_data_path}

eval_data.npy  eval_labels.npy	train_data.npy	train_labels.npy


In [15]:
bucket = sagemaker_session.default_bucket()
data_prefix = 'sagemaker/tensorflow-mnist/data'

In [36]:
training_data_uri = sagemaker_session.upload_data(path=local_data_path, bucket=bucket, key_prefix=data_prefix)
print(training_data_uri)

s3://sagemaker-us-east-1-835319576252/tensorflow-mnist/data


In [37]:
!aws s3 ls --recursive s3://{bucket}/{data_prefix}

2019-12-26 17:47:12   31360128 tensorflow-mnist/data/eval_data.npy
2019-12-26 17:47:12      40128 tensorflow-mnist/data/eval_labels.npy
2019-12-26 17:47:12  172480128 tensorflow-mnist/data/train_data.npy
2019-12-26 17:47:12     220128 tensorflow-mnist/data/train_labels.npy


# Distributed Keras + TensorFlow 2.x Training on SageMaker

https://sagemaker.readthedocs.io/en/stable/using_tf.html#distributed-training

This tutorial's training script was adapted from TensorFlow's official [CNN MNIST example](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/layers/cnn_mnist.py). We have modified it to handle the ``model_dir`` parameter passed in by SageMaker. This is an S3 path which can be used for data sharing during distributed training and checkpointing and/or model persistence. We have also added an argument-parsing function to handle processing training-related variables.

At the end of the training job we have added a step to export the trained model to the path stored in the environment variable ``SM_MODEL_DIR``, which always points to ``/opt/ml/model``. This is critical because SageMaker uploads all the model artifacts in this folder to S3 at end of training.

## Define the Keras + TensorFlow 2.x Script

In [22]:
!ls 'src/mnist_keras_tf2.py'

src/mnist_keras_tf2.py


## Create a training job using the `TensorFlow` estimator

https://sagemaker.readthedocs.io/en/stable/using_tf.html#distributed-training

The `sagemaker.tensorflow.TensorFlow` estimator handles locating the script mode container, uploading your script to a S3 location and creating a SageMaker training job. Let's call out a couple important parameters here:

* `py_version` is set to `'py3'` to indicate that we are using script mode since legacy mode supports only Python 2. Though Python 2 will be deprecated soon, you can use script mode with Python 2 by setting `py_version` to `'py2'` and `script_mode` to `True`.

* `distributions` is used to configure the distributed training setup. It's required only if you are doing distributed training either across a cluster of instances or across multiple GPUs. Here we are using parameter servers as the distributed training schema. SageMaker training jobs run on homogeneous clusters. To make parameter server more performant in the SageMaker setup, we run a parameter server on every instance in the cluster, so there is no need to specify the number of parameter servers to launch. Script mode also supports distributed training with [Horovod](https://github.com/horovod/horovod). You can find the full documentation on how to configure `distributions` [here](https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#distributed-training). 

Notes:  
* This example uses two(2) `ml.p3.2xlarge` instances.  You will likely need to request a SageMaker instance limit increase from Support before continuing.

* Alternatively, you can specify `ml.c5.2xlarge` instead.

* To recognize the `requirements.txt`, we must include `src/setup.py` per [this](https://github.com/aws/sagemaker-python-sdk/issues/911) GitHub issue.

In [16]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist_keras_tf2.py',
                             source_dir='./src',
                             role=role,
                             train_instance_count=2,
                             train_instance_type='ml.p3.2xlarge',
                             framework_version='2.0.0',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}})

## Calling `fit` (Approx. 15 mins)

To start a training job, we call `estimator.fit(training_data_uri)`.

An S3 location is used here as the input. `fit` creates a default channel named `'training'`, which points to this S3 location. In the training script we can then access the training data from the location stored in `SM_CHANNEL_TRAINING`. `fit` accepts other parameters, as well. See the API doc [here](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) for details.

When training starts, the TensorFlow container executes `mnist_keras_tf2.py` by passing `hyperparameters` and `model_dir` from the estimator as script arguments. Because we didn't define either in this example, no hyperparameters are passed, and `model_dir` defaults to `s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`, so the script execution is as follows:
```bash
python mnist_keras_tf2.py --model_dir s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>
```
When training is complete, the training job will upload the saved model for TensorFlow serving.

In [17]:
# TODO:  `training_data_uri` causes a `//` char error

mnist_estimator.fit(inputs=training_data_uri, wait=False)

#mnist_estimator.fit(inputs=original_training_data_uri, wait=False)
training_job_name = mnist_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

training_job_name:  tensorflow-training-2019-12-25-17-45-08-671


After some time, or in a separate Python notebook, we can attach to the running job using the `training_job_name`.

In [ ]:
mnist_estimator = TensorFlow.attach(training_job_name=training_job_name)

2019-12-25 17:45:10 Starting - Launching requested ML instances......
2019-12-25 17:46:12 Starting - Preparing the instances for training.........
2019-12-25 17:47:24 Downloading - Downloading input data...
2019-12-25 17:48:11 Training - Training image download completed. Training in progress...2019-12-25 17:48:15,256 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-12-25 17:48:15,574 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2019-12-25 17:48:15,574 sagemaker_tensorflow_container.training INFO     Launching parameter server process
2019-12-25 17:48:15,574 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2019-12-25 17:48:16,142 sagemaker_tensorflow_container.training INFO     Launching worker process
2019-12-25 17:48:16,250 sagemaker-containers INFO     Installing module with the following command:
/usr/bin/python3 -m pip ins

# Option 1:  Perform Batch Predictions Directly in the Notebook

In [19]:
# TODO

# Option 2:  Perform Batch Predictions with a Batch Transform Job

In [20]:
# TODO

We can create a [Batch Transformation Job](https://sagemaker.readthedocs.io/en/stable/using_tf.html#run-a-batch-transform-job) to run batch predictions.

In [ ]:
batch_output = '{}/batch_output'.format(training_data_uri) # The location to store the results

mnist_transformer = mnist_estimator.transformer(instance_count=1, instance_type='ml.p3.2xlarge', output_path=batch_output)

### TODO:  We may need to use a custom data transformer to convert numpy format into json

In [ ]:
 mnist_transformer.transform(data=training_data_uri, data_type='S3Prefix', content_type='text/json', split_type='Line')

# Option 3:  Create a SageMaker Endpoint and Perform REST-based Predictions

In [21]:
# TODO

# Deploy the Trained Model to a SageMaker Endpoint (Approx. 10 mins)

The `deploy()` method creates a SageMaker model, which is then deployed to an endpoint to serve prediction requests in real time. We will use the TensorFlow Serving container for the endpoint, because we trained with script mode. This serving container runs an implementation of a web server that is compatible with SageMaker hosting protocol. The [Using your own inference code](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) document explains how SageMaker runs inference containers.

In [ ]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

# Invoke the endpoint

Let's download the training data and use that as input for inference.

In [ ]:
import numpy as np

train_data = np.load('data/train_data.npy')
train_labels = np.load('data/train_labels.npy')

The formats of the input and the output data correspond directly to the request and response formats of the `Predict` method in the [TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest). SageMaker's TensforFlow Serving endpoints can also accept additional input formats that are not part of the TensorFlow REST API, including the simplified JSON format, line-delimited JSON objects ("jsons" or "jsonlines"), and CSV data.

In this example we are using a `numpy` array as input, which will be serialized into the simplified JSON format. In addition, TensorFlow Serving can also process multiple items at once as you can see in the following code. You can find the complete documentation on how to make predictions against a TensorFlow Serving SageMaker endpoint [here](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint).

Examine the prediction result from the TensorFlow 2.0 model.

In [ ]:
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = predictions['predictions'][i]
    label = train_labels[i]
    print('prediction is {}, label is {}, matched: {}'.format(prediction, label, prediction == label))

# Delete the endpoint

Let's delete the endpoint we just created to prevent incurring any extra costs.

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)